# Tutorial Basics Noten Part 2: Statistik

Inhaltsverzeichnis: 
1. Abfrage von einfachen statistischen Angaben
2. Häufigkeiten: Tonhöhen, Tonhöhenklassen, Tondauern, metrische Akzente, Intervalle
3. Zweidimensionale Häufigkeitsverteilungen
4. Aufgaben

In diesem Tutorial werden Sie anhand von Notenbeispielen einfache computergestützten Möglichkeiten statistischer Abfragen auf der Grundlage von CAMAT kennenlernen. 

Das Durcharbeiten und Nachvollziehen des Tutoriums soll es Ihnen ermöglichen, mit den vorgestellten computergestützten Methoden eigene Musikbeispiele (Notendateien) zu untersuchen und Vergleiche zwischen verschiedenen Musikstücken durchzuführen. 
Dabei geht es immer auch um die Formulierung bestimmter analytischer Fragestellungen und die Interpretation der jeweiligen Ergebnisse.

Jeder Start eines Jupyter Notebooks beginnt mit dem Import einer Reihe von Python-Bibliotheken, die für die Analyse erforderlich sind:  

In [1]:
import sys
import os
sys.path.append(os.getcwd().replace(os.path.join('music_xml_parser', 'ipynb'), ''))

import music_xml_parser as mp
import csv
from IPython.display import HTML, display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Mit diesen Befehlen wird nicht nur 'music21', 
# sondern darüber hinaus die Bibliotheken 'numpy' und 'pandas' für statische Auswertungen 
# sowie 'matplotlib' für grafische Darstellungen geladen. 

# mit den folgenden Befehlen wird die Formtierung für die Tabellen festgelegt, 
# die weiter unten dargestellt werden: 

pd.set_option('display.max_rows', 9999)
pd.set_option('display.max_columns', 9999)
pd.set_option('display.width', 9999)






ModuleNotFoundError: No module named 'parser_utils'

Anschließend müssen Sie die Datei laden, die Sie untersuchen wollen (aus dem Internet oder von Ihrer Festplatte bzw. aus dem Notenordner) und den xml-parser aktivieren. Dabei wird aus der xml-Datei ein neuer dataframe ('m_df') erzeugt, der den folgenden statistischen Abfragen zugrunde liegt. 

Wir wählen für unser Tutorial als Beispiel den ersten Satz aus dem Streichquartett KV. 171 von Wolfgang Amadeus Mozart. 

In [ ]:
# xml_file = 'PrJode_Jos1102_COM_1-5_MissaLasol_002_00137.xml'
#xml_file = 'BaJoSe_BWV18_COM_5-5_CantataGle_004_00110.xml'
# xml_file = 'BeLuva_Op59_1-3_1-4_StringQuar_003_00129.xml'

#xml_file = 'MoWo_K279_COM_1-3_PianoSonat_003_00920.xml'
#xml_file = 'https://analyse.hfm-weimar.de/database/03/BoMaEn_Op115_11-22_COM_ThemeetVar_003_00181.xml'
# xml_file = 'https://analyse.hfm-weimar.de/database/03/MoWo_K279_COM_1-3_PianoSonat_003_00920.xml'
xml_file = 'https://analyse.hfm-weimar.de/database/03/MoWo_K171_COM_1-4_StringQuar_003_00867.xml'

m_df = mp.parse.with_xml_file(file=xml_file,
                                    save_at=None,
                                    save_file_name=None,
                                    do_save=False, 
                                    x_axis_res=2)

# 'm_df' wird als Variable für den Pandas-Dataframe definiert. 
# Dabei können verschiedene arameter festgelegt werden (in der Regel mit True/False oder True/None)

# save_at 
# save_file_name
# do_save
# x_axis_res - Auflösung der X-Achse als Teiler der Viertelnote (2=0.5 Viertelnote)
# get_measure_onset 

@ Egor: Könntest Du bitte noch die einzelnen Parameter kurz erläutern? (Oder löschen, falls Sie hier nicht relevant sind. 

### 1. Abfrage von einfachen statistischen Angaben

Wir beginnen mit einfachen statistischen Abfragen: Anzahl der Stimmen, Anzahl der Töne (insgesamt und pro Stimmen), Länge in Takten, Ambitus:

In [ ]:
v = m_df[['PartID','PartName']].drop_duplicates().to_numpy()
mp.utils.display_table(data=v, 
                       columns=['Part ID', 'PartNames'])

# in der ersten Zeile wird aus der dataframe-Liste des xml-Parser ('m_df')
# die PartIDs und PartNames, 
# also die IDs und Bezeichnungen der einzelnen Stimmen, abgefragt. 
# Hierfür wird die Variable 'v' vergeben.
# In der zweiten Zeile wird die Darstellung der Tabelle festgelegt. 

In [ ]:
m = m_df['Measure'].to_numpy(dtype=int)
max(m)
# Abfrage der Stückänge in Takten

In [ ]:
n_notes, c_notes = np.unique(m_df['PartID'], return_counts=True)
data = [[i, c] for i, c in zip(n_notes, c_notes) ]
mp.utils.display_table(data=data, 
                       columns=['PartID', '# Notes'])
 
# Abfrage der Anzahl der Töne per Stimme 
# (wobei übergebundene Noten jeweils als eine Note gezählt werden). 
# Falls die Stimmen verschiedene Bezeichnungen besitzen, 
# kann 'Part ID' durch 'Part Name' ersetzt werden. 

In [ ]:
ambitus = mp.analyse.ambitus(m_df,output_as_midi=True)
mp.utils.display_table(data=ambitus, 
                       columns=['Part ID', 'min', 'max', 'Ambitus'])

# Der Ambitus per Stimme errechnet sich aus der Differenz zwischen 
# dem niedrigsten (min) und höchsten (max) Ton, hier angegeben in MIDI-Werten. 

In [ ]:
ambitus = mp.analyse.ambitus(m_df,output_as_midi=False)
mp.utils.display_table(data=ambitus, 
                       columns=['Part ID', 'min', 'max', 'Semitones Difference'])

# Durch den Zusatz 'output_as_midi=False' werden die Töne mit Tonnamen angegeben. 

Aufgabe: Ersetzen Sie in dem Befehl 'Part ID' durch 'Part Name' und drücken Sie im Menu auf Run. Wie verändert sich dadurch die Tabelle?

### 2. Häufigkeiten: Tonhöhen, Tonhöhenklassen, Tondauern

Zur Charakterisierung einzelner Musikstücke und zum Vergleich zwischen verschiedenen Musikstücken kann es sinnvoll sein, die Häufigkeit bestimmter Elemente (Tonhöhen, Dauernwerte, Akkorde etc.) zu bestimmen. Für solche Fragen lassen sich Häufigkeitstabellen und grafische Darstellungen, sog. Histogramme erstellen. 

### 2.1 Tonhöhen

Welche Töne tauchen wie häufig auf? Wie diatonisch ist der Tonraum der Komposition, wie viele zusätzliche chromatische Töne tauchen auf?

In [ ]:

pitch_hist = mp.analyse.pitch_histogram(m_df, do_plot=True, visulize_midi_range=None)
mp.utils.display_table(data=pitch_hist, 
                       columns=['MIDI', 'Pitch','Occurences'])
# Mit der ersten Zeile wird die Abfrage ausgeführt 
# und die Grafik dargestellt ('do_plot=True').
# Mit der zweiten Zeile wird die Tabelle dargestellt und beschriftet. 


Was können wir beobachten?

##### Vertiefungen:

Die Grafik lässt sich auch in einem externen Pop-Up-Fenster des Programms Matplotlib darstellen und dort weiterverarbeiten, speichern etc. Dazu muss dem Code der Befehl '%matplotlib' vorangestellt werden: 

In [ ]:
%matplotlib
pitch_hist = mp.analyse.pitch_histogram(m_df, do_plot=True, visulize_midi_range=None)

Anschließend muss Matplotlib durch den Befehl '%matplotlib inline' wieder ausgeschaltet werden. Ansonsten werden alle folgenden Grafiken extern dargestellt. 

In [ ]:
%matplotlib inline

Es gibt zwei weitere Möglichkeiten, die Darstellung zu verbessern: 
Einerseits kann die Darstellung auf einen bestimmten Tonhöhenbereich eingeschränkt werden. 
Andererseits können nur jene Tonhöhen ausgewählt werden, die tatsächlich auftreten. 

In [ ]:
ph = mp.analyse.pitch_histogram(m_df, do_plot=True, visulize_midi_range=[50, 90])

# Durch den Zusatz 'visulize_midi_range=[50, 90]' wurde der dargestellte Ausschnitt
# auf den Bereich zwischen MIDI-Pitch 50 (= D3) und 90 (= F6) eingegrenzt.

In [ ]:

ph2 = mp.analyse.pitch_histogram(m_df, do_plot=True, do_plot_full_axis=False, visulize_midi_range=None)

# Durch den Zusatz 'do_plot_full_axis=False,' werden bei der folgenden Grafik 
# nur die Häufigkeiten von Tönen angezeigt, die auch tatsächlich auftauchen. 
# Alle anderen Töne werden auf der x-Achse gelöscht. 

Durch den folgenden Befehl wird eine Liste der Tonhöhenhäufigkeiten angezeigt und als csv-Datei exportiert (csv = comma separated variables; lesbar und weiterverarbeitbar u.a. in Excel oder im Text-Editor). Durch den Export lassen sich Tabellen zum Stückvergleich und zur Korpusanalyse generieren. Die csv-Datei wird in dem Ordner gespeichert, den Sie in Ihren Settings als "settings['directoryScratch']" festgelegt haben und kann mit einem Texteditor oder einem Tabellenkalkulationsprogramm (z.B. Excel) geöffnet werden. 

In [ ]:
mp.utils.export_as_csv(data=pitch_hist, 
                       columns=['MIDI','Pitch','Occurrences'],
                       save_file_name ='pitch_histogram.csv',  # Sie können auch einen anderen Dateinamen festlegen.
                       do_save=True,    # Befehl für die Speicherung
                       do_print=False,  # bei 'True' wird die Datei nochmals im Browser angezeigt.
                       do_return_pd=False,
                       sep=';',         # Als Trennzeichen wird ein Semikolon verwendet. 
                       index=False, 
                       header=True)

# Die Datei pitch_histogram.csv wird automatisch im Ordner music_xml_parser\data\exports\ gespeichert. 

# "do_save=True" nicht vergessen, dann wird csv im demselben Ordner wie der Jupyter-Notebook gespeichert
# Wenn Sie in einem anderen Ordner abspeichern wollen, so müssen Sie unter 
# save_file_name einen Pfad eingeben (z.B. 'C:/pitch_histogram.csv')

@Egor: Bitte noch die restlichen Variablen (do_return_pd, index, header) kurz erläutern!!

### 2.2 Tonhöhenklassen

Die Tönhöhen lassen sich in einem zweiten Schritt zu Tonhöhenklassen zusammenfassen. Dadurch wird das Ergebnis übersichtlicher. 

In [ ]:
pitchclass_hist = mp.analyse.pitch_class_histogram(m_df, do_plot=True)

# Die Häufigkeiten werden mit dem folgenden Befehl abgespeichert: 

mp.utils.export_as_csv(data=pitchclass_hist, 
                       columns=['Pitch Class','Occurrences'],
                       save_file_name ='pitch_class_hist.csv', 
                       do_save=False,
                       do_print=True, 
                       do_return_pd=False,
                       sep=';', 
                       index=False, 
                       header=True)


To do: Interpretation?

### 2.3 Tondauern 

Welche Dauernwerte werden in der Komposition verwendet und wie häufig kommen diese jeweils vor?

In [ ]:
# Für das Dauernwerte-Histogram muss eine andere Befehlsmethode angewendet werden.
# Der graph-Befehl muss mit dem run-Befehl kombiniert werden. 

quarter_dur_hist = mp.analyse.quarterlength_duration_histogram(m_df,
                                                               with_pitch=False, 
                                                               with_pitchclass=False,
                                                               do_plot=True)

mp.utils.export_as_csv(data=quarter_dur_hist, 
                       columns=['Duration', 'Occurrences'],
                       save_file_name ='quarter_duration_hist.csv', 
                       do_save=False,         
                       do_print=True, 
                       do_return_pd=False,
                       sep=';', 
                       index=False, 
                       header=True)

Die Notenwerte werden als Vielfache und Bruchteile einer Viertelnote (Quarter Length) gezählt. 

@ Christon: how could the bar chart be adjusted in a way that the bars (for the quarter length subdivisions) don't overlap?

### 2.4 Metrisches Profil 

Wie deutlich wird das Metrum in den einzelnen Stimmen einer Komposition – durch die Platzierung der Töne auf Taktanfängen und den metrisch wichtigen Positionen innerhalb des Taktes (z.B. der Taktmitte oder auf den Viertelpositionen)? Hierzu lässt sich in eine Liste der Häufigkeiten von Tönen auf den verschiedenen metrischen Positionen in den einzelnen Stimmen anzeigen. 
Ein solches Profil hat natürlich zur Voraussetzung, dass das untersuchte Stück in einem einzigen Metrum steht und keine Taktwechsel hat. Das überprüft man mit dem folgenden Befehl: 

In [ ]:
ts_hist = mp.analyse.time_signature_histogram(m_df, do_plot=False,do_adjusted=False)
mp.utils.display_table(data=ts_hist, 
                       columns=['Time Signature', 'Occurences'])

@ Christon: Is it possible to list not occurences (= note elements?) but measures instead?

Es tauchen also sowohl 4/4-Takte als auch 3/4-Takte in dem Stück auf, wobei der 3/4-Takt überwiegt. Mit dem folgenden Befehl werden daher zwei unterschiedliche metrische Profile erzeugt. 

In [ ]:
bs_hist_list = mp.analyse.metric_profile_split_time_signature(m_df, with_pitch=False, do_plot=True)



### 2.4 Intervalle im einstimmigen Verlauf

Wie häufig kommen bestimmte Intervallschritte in den einzelnen Stimmen vor? Sind da alle Stimmen ähnlich gelagert – oder gibt es z.B. in den Unterstimmen mehr Sprünge, in der Melodiestimme mehr Schritte?

Schauen wir uns zunächst die Intervallverteilung in der ersten Stimme an. 

In [ ]:
v = m_df[['PartID','PartName']].drop_duplicates().to_numpy()
print(v)

# Durch diesen Befehl werden zunächst die Stimmen und ihre Bezeichnung angezeigt. 

interval_hist = mp.analyse.interval(m_df,
                                    part='1',
                                    do_plot=True)

# Durch 'part='1'' wird die erste Stimme angezeigt. 
# Der folgende Befehl exportiert die Intervallhäufigkeiten
# in die Datei 'interval_1.csv' (nur falls 'do_save=True')

mp.utils.export_as_csv(data=interval_hist, 
                       columns=['Interval', 'Occurrences'],
                       save_file_name ='interval_1.csv', 
                       do_save=False,
                       do_print=True, 
                       do_return_pd=False,
                       sep=';', 
                       index=False, 
                       header=True)

//To Do: Interpretation//

### 3. Zweidimensionale Häufigkeitsverteilungen

Wir haben uns bereits die Häufigkeiten von Tonhöhen bzw. Tonhöhenklassen angeschaut. Nun könnte man sagen: Längere Töne haben natürlich mehr Gewicht als kurze Töne oder Töne zwischen den Zählzeiten. Diesen Gedanken können wir weiterverfolgen, indem wir uns kombinierte, ‚doppelte‘ oder ‚bivariate‘ Häufigkeitsverteilungen: Also z.B. die Häufigkeiten der Tonhöhen für jeweils die verschiedenen Dauernwerte oder die Häufigkeiten der Tonhöhenklasse jeweils für die verschiedenen metrischen Positionen. Hierum soll es im Folgenden anhand von zwei Beispielen gehen.  

Beispiel 1: Dauern pro Tonhöhenklassen.
Gibt es Unterschiede bei den Dauernwerten bzgl. der verschiedenen Tonhöhenklassen?

Beispiel 2: Tonhöhen auf metrischen Positionen
Gibt es Unterschiede bei den Dauernwerten bzgl. der verschiedenen Tonhöhenklassen?

@ Christon: How can we choose pitch classes instead of pitches (in section 3.1 and 3.2). Pitch classes are much more easily understandable...

### 3.1 Dauernwerte pro Tonhöhen

In [ ]:
dur_pc_hist = mp.analyse.quarterlength_duration_histogram(m_df,
                                                          with_pitch=True,
                                                          with_pitchclass=False,
                                                          do_plot=True)
# Durch den Zusatz 'with_pitch=True' wird eine 3D-Grafik der 
# Tonhöhen pro Dauernklasse aktiviert. 

Da die CD-Grafik im Browser sehr klein ist, empfiehlt es sich auch hier, sie in einem externen Fenster zu öffnen. Dies geschieht, indem der Befehl '%matplotlib' vorangestellt wird: 

In [ ]:
%matplotlib

dur_pc_hist = mp.analyse.quarterlength_duration_histogram(m_df,
                                                          with_pitch=True,
                                                          with_pitchclass=False,
                                                          do_plot=True)

Nun öffnet sich die Grafik im Matplotlib-Pop-Up-Fenster und kann dort weiter bearbeitet, gedreht und vergrößert werden. Um diese Funktion bei folgenden Grafiken wieder auszuschalten, muss der Befehl '%matplotlib inline' ausgeführt werden: 

In [ ]:
%matplotlib inline

### 3.2 Tonhöhen pro metrischer Position

In [ ]:
metric_profile_p_hist = mp.analyse.metric_profile(m_df, with_pitch=True, do_plot=True)

mp.utils.export_as_csv(data=metric_profile_p_hist, 
                       columns=['MIDI', 'Pitch','Metric Profile','Occurrences'],
                       save_file_name ='metric_profile_hist.csv', 
                       do_save=False,
                       do_print=True, 
                       do_return_pd=False,
                       sep=';', 
                       index=False, 
                       header=True)

Auch hier ist es sinnvoll, sich die Grafik im externen Pop-Up-Fenster von Matplotlib anzuschauen. 

In [ ]:
%matplotlib

# Wichtig: Matplotlib muss anschließend durch die #-Taste sowie zusätzlich
# durch den Befehl '&matplotlib inline' wieder ausgeschaltet werden, 
# sonst werden alle folgenden Grafiken im externen Fenstergeöffnet!

metric_profile_p_hist = mp.analyse.metric_profile(m_df, with_pitch=True, do_plot=True)

mp.utils.export_as_csv(data=metric_profile_p_hist, 
                       columns=['MIDI', 'Pitch','Metric Profile','Occurrences'],
                       save_file_name ='metric_profile_hist.csv', 
                       do_save=False,
                       do_print=True, 
                       do_return_pd=False,
                       sep=';', 
                       index=False, 
                       header=True)

In [ ]:
bs_3dhist_list = mp.analyse.metric_profile_split_time_signature(m_df, with_pitch=True, do_plot=True)


In [ ]:
mp.utils.display_table(data=bs_3dhist_list[0], 
                       columns=['MIDI', 'Pitch', 'Metric Profile', 'Occurrences'])

### 4. Anwendung von Takt- und Stimmenfilter

Alle statistischen Abfragen lassen sich durch eine einfach zu bedienende Filterfunktion auf beliebige Ausschnitte und Stimmen eingrenzen. Dazu muss man nur die entsprechenden Taktangaben und Stimmenbezeichnungen eingeben. 

Hier zwei Beispiele: 

Mit den folgenden Befehlen lässt sich die Liste der Intervallhäufigkeiten anzeigen exportieren bzw. als Histogramm anzeigen.

In [ ]:
# Ausgewählt werden die erste und zweite Stimme ('PartID':'1-2')
# und die Takte 1-5 ('Measure':'1-5')
# Diese Auswahl lässt sich beliebig verändern! 

filter_dict_interval ={'PartID':'1-2', 'Measure':'1-5'}
interval_hist_example = mp.analyse.interval(m_df,
                                    part='all',
                                    do_plot=True, filter_dict = filter_dict_interval)

mp.utils.export_as_csv(data=interval_hist_example, 
                       columns=['Interval', 'Occurrences'],
                       save_file_name ='interval.csv', 
                       do_save=False,
                       do_print=True, 
                       do_return_pd=False,
                       sep=';', 
                       index=False, 
                       header=True)

### 5. Aufgaben

Vergleichende statistische Abfragen über mehrere Stücke:

Wir haben uns die Ergebnisse bislang nur anhand jeweils eines Stückes angeschaut. Wie stellt sich nun aber die Lage dar, wenn wir mehrere Stücke, z.B. mehrere oder alle Sätze der Komposition, miteinander – und mit anderen Stücken – vergleichen? Gibt es da tatsächlich stilistische Regelmäßigkeiten – oder überwiegen die Unterschiede?

Wählen Sie Kompositionen Ihrer Wahl (verschiedene Gattungen, Komponisten und Epochen) und vergleichen Sie diese Stücke hinsichtlich der Häufigkeiten von Tonhöhen, Tonhöhenklassen, Notenwerten und Intervallen. Beschreiben Sie die Ergebnisse!